<a href="https://colab.research.google.com/github/YuxingW/machine_learning/blob/main/HW4_mullerloop/muller_loop_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### What is the story

This note book is to predict the instant power consumption based on smart home sensor data and latent variables <br />
Dataset<br />
&nbsp;  &nbsp; &nbsp; Home C sensor data set in 2015<br />

Objective fuction<br />
Predict instant power consumption based on sensor data solar energy and latent variables -- weather info<br />



In [5]:
import requests
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from io import BytesIO

Load the dataset which has aggregated the sensor power consumption data combined with weather data of Home C.

In [6]:
train_url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vTAH13xa_KY8QeIy48jCgbAY7OkFQCQYfHh-7Nt-TgXKTBeH8buWJZ6Izm91LxIXCo9W9oebwmbgbeF/pub?output=csv#'
r = requests.get(train_url)
data = r.content
stats = pd.read_csv(BytesIO(data))
stats.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (27) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,time,use [kW],gen [kW],House overall [kW],Dishwasher [kW],Furnace 1 [kW],Furnace 2 [kW],Home office [kW],Fridge [kW],Wine cellar [kW],Garage door [kW],Kitchen 12 [kW],Kitchen 14 [kW],Kitchen 38 [kW],Barn [kW],Well [kW],Microwave [kW],Living room [kW],Solar [kW],temperature,icon,humidity,visibility,summary,apparentTemperature,pressure,windSpeed,cloudCover,windBearing,precipIntensity,dewPoint,precipProbability
0,1451624400,0.932833,0.003483,0.932833,0.000033,0.020700,0.061917,0.442633,0.124150,0.006983,0.013083,0.000417,0.000150,0.000000,0.031350,0.001017,0.004067,0.001517,0.003483,36.14,clear-night,0.62,10.0,Clear,29.26,1016.91,9.18,cloudCover,282,0.0,24.4,0.0
1,1451624401,0.934333,0.003467,0.934333,0.000000,0.020717,0.063817,0.444067,0.124000,0.006983,0.013117,0.000417,0.000150,0.000000,0.031500,0.001017,0.004067,0.001650,0.003467,36.14,clear-night,0.62,10.0,Clear,29.26,1016.91,9.18,cloudCover,282,0.0,24.4,0.0
2,1451624402,0.931817,0.003467,0.931817,0.000017,0.020700,0.062317,0.446067,0.123533,0.006983,0.013083,0.000433,0.000167,0.000017,0.031517,0.001000,0.004067,0.001650,0.003467,36.14,clear-night,0.62,10.0,Clear,29.26,1016.91,9.18,cloudCover,282,0.0,24.4,0.0
3,1451624403,1.022050,0.003483,1.022050,0.000017,0.106900,0.068517,0.446583,0.123133,0.006983,0.013000,0.000433,0.000217,0.000000,0.031500,0.001017,0.004067,0.001617,0.003483,36.14,clear-night,0.62,10.0,Clear,29.26,1016.91,9.18,cloudCover,282,0.0,24.4,0.0
4,1451624404,1.139400,0.003467,1.139400,0.000133,0.236933,0.063983,0.446533,0.122850,0.006850,0.012783,0.000450,0.000333,0.000000,0.031500,0.001017,0.004067,0.001583,0.003467,36.14,clear-night,0.62,10.0,Clear,29.26,1016.91,9.18,cloudCover,282,0.0,24.4,0.0


### Data preprosessing
Remove unit(kW) from dataset column names.

In [7]:
stats.columns = [i.replace(' [kW]', '') for i in stats.columns]

'cloudCover' value was found in the column, let's replace these invalid values with the next valid value.

In [8]:
stats['cloudCover'].replace(['cloudCover'], method='bfill', inplace=True)
stats['cloudCover'] = stats['cloudCover'].astype('float')
stats.head()

,time,use,gen,House overall,Dishwasher,Furnace 1,Furnace 2,Home office,Fridge,Wine cellar,Garage door,Kitchen 12,Kitchen 14,Kitchen 38,Barn,Well,Microwave,Living room,Solar,temperature,icon,humidity,visibility,summary,apparentTemperature,pressure,windSpeed,cloudCover,windBearing,precipIntensity,dewPoint,precipProbability
0,1451624400,0.932833,0.003483,0.932833,0.000033,0.020700,0.061917,0.442633,0.124150,0.006983,0.013083,0.000417,0.000150,0.000000,0.031350,0.001017,0.004067,0.001517,0.003483,36.14,clear-night,0.62,10.0,Clear,29.26,1016.91,9.18,0.75,282,0.0,24.4,0.0
1,1451624401,0.934333,0.003467,0.934333,0.000000,0.020717,0.063817,0.444067,0.124000,0.006983,0.013117,0.000417,0.000150,0.000000,0.031500,0.001017,0.004067,0.001650,0.003467,36.14,clear-night,0.62,10.0,Clear,29.26,1016.91,9.18,0.75,282,0.0,24.4,0.0
2,1451624402,0.931817,0.003467,0.931817,0.000017,0.020700,0.062317,0.446067,0.123533,0.006983,0.013083,0.000433,0.000167,0.000017,0.031517,0.001000,0.004067,0.001650,0.003467,36.14,clear-night,0.62,10.0,Clear,29.26,1016.91,9.18,0.75,282,0.0,24.4,0.0
3,1451624403,1.022050,0.003483,1.022050,0.000017,0.106900,0.068517,0.446583,0.123133,0.006983,0.013000,0.000433,0.000217,0.000000,0.031500,0.001017,0.004067,0.001617,0.003483,36.14,clear-night,0.62,10.0,Clear,29.26,1016.91,9.18,0.75,282,0.0,24.4,0.0
4,1451624404,1.139400,0.003467,1.139400,0.000133,0.236933,0.063983,0.446533,0.122850,0.006850,0.012783,0.000450,0.000333,0.000000,0.031500,0.001017,0.004067,0.001583,0.003467,36.14,clear-night,0.62,10.0,Clear,29.26,1016.91,9.18,0.75,282,0.0,24.4,0.0


Combine 'Furnace 1' and 'Furnace 2' to 'Furnace', combine 'Kitchen 12' 'Kitchen 14' and 'Kitchen 38' to 'Kitchen'.

In [9]:
stats['Furnace'] = stats[['Furnace 1','Furnace 2']].sum(axis=1)
stats['Kitchen'] = stats[['Kitchen 12','Kitchen 14','Kitchen 38']].sum(axis=1)

See feature correlations, Solar data is negatively correlated to power consumption, that is expected.

In [10]:
  feature = 'use'
  df_corr = stats.corr()[feature]
  golden_features_list = df_corr[abs(df_corr) > 0.1].sort_values(ascending=True)
  print("There is {} correlated values with feature {}:\n{}"
        .format(len(golden_features_list), feature, golden_features_list))

There is 21 correlated values with feature use:
gen                   -0.309440
Solar                 -0.309440
time                  -0.159069
apparentTemperature   -0.154378
temperature           -0.152890
dewPoint              -0.120681
Kitchen 38             0.107878
Kitchen 14             0.154937
Home office            0.155206
Microwave              0.158953
Kitchen                0.170290
Fridge                 0.188517
Barn                   0.239522
Well                   0.257693
Dishwasher             0.302407
Furnace 1              0.329671
Living room            0.342365
Furnace 2              0.483417
Furnace                0.543849
House overall          1.000000
use                    1.000000
Name: use, dtype: float64


Observe duplicate feature 'use' and 'House overall', 'gen' and 'Solar', remove one of them. <br />
Keep only features which absolute correlation with 'use' > 0.1, not involving duplicate features.

In [11]:
feature_list = ['gen' , 'apparentTemperature', 'temperature', 'dewPoint', 'Kitchen', 'Furnace', \
                'Home office', 'Microwave', 'Fridge', 'Barn', 'Well', 'Dishwasher', 'Living room', 'use']
df = stats[feature_list]
df.head()

,gen,apparentTemperature,temperature,dewPoint,Kitchen,Furnace,Home office,Microwave,Fridge,Barn,Well,Dishwasher,Living room,use
0,0.003483,29.26,36.14,24.4,0.000567,0.082617,0.442633,0.004067,0.124150,0.031350,0.001017,0.000033,0.001517,0.932833
1,0.003467,29.26,36.14,24.4,0.000567,0.084533,0.444067,0.004067,0.124000,0.031500,0.001017,0.000000,0.001650,0.934333
2,0.003467,29.26,36.14,24.4,0.000617,0.083017,0.446067,0.004067,0.123533,0.031517,0.001000,0.000017,0.001650,0.931817
3,0.003483,29.26,36.14,24.4,0.000650,0.175417,0.446583,0.004067,0.123133,0.031500,0.001017,0.000017,0.001617,1.022050
4,0.003467,29.26,36.14,24.4,0.000783,0.300917,0.446533,0.004067,0.122850,0.031500,0.001017,0.000133,0.001583,1.139400


In [12]:
df.shape

(113176, 14)

### Apply Mullar Loop
Apply MLP, LinearRegression, RandomForestRegressor, KNNRegressor, XBoost Regressors and evaluate the learnings with r2_score and MSE.

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import xgboost as xgb
from time import time

In [14]:
X, y = df.drop(['use'], axis=1), df['use']

X = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=.2, shuffle=True)

In [25]:
def muller_loop(X_train, y_train, X_test, y_test):
  names = ['Linear Regression', 
           'Random Forest',
           'Decision Tree',
           'MLP Regressor',
           'XBoost Regressor',
           'KNN Regressor'
          ]
  regression = [
            LinearRegression(),
            RandomForestRegressor(max_depth=5, n_estimators=10),
            DecisionTreeRegressor(max_depth=5),
            MLPRegressor(random_state=1, alpha=1, max_iter=1000),
            xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10),
            KNeighborsRegressor(3),
          ]
  result = pd.DataFrame(columns=['model','r2 score', 'mean squared error', 'mean absolute error', 'time'])
  # TODO (Apply): All cross-validation

  max_score = 0.0
  max_class = ''
  # iterate over classifiers
  for name, clf in zip(names, regression):
      start_time = time()
      clf.fit(X_train, y_train)
      r2 = 100.0 * clf.score(X_test, y_test)
      y_pred = clf.predict(X_test)
      v_score = 100.0 * explained_variance_score(y_test, y_pred, multioutput='raw_values')[0]
      mse =  mean_squared_error(y_test, y_pred)
      mae =  mean_absolute_error(y_test, y_pred)
      result = result.append({'model': name, 'r2 score': r2, 'mean squared error': mse, 'mean absolute error': mae ,'time': (time() - start_time) }, ignore_index=True)

      if r2 > max_score:
          clf_best = clf
          max_score = r2
          max_class = name

  display(result)

In [26]:
muller_loop(X_train, y_train, X_test, y_test)

[04:51:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


,model,r2 score,mean squared error,mean absolute error,time
0,Linear Regression,75.371517,0.116378,0.153765,0.044595
1,Random Forest,63.283852,0.173496,0.242867,3.010132
2,Decision Tree,61.176183,0.183456,0.248552,0.460096
3,MLP Regressor,77.052136,0.108436,0.143961,10.666155
4,XBoost Regressor,40.127532,0.282918,0.323179,0.696721
5,KNN Regressor,88.140586,0.056040,0.076253,36.435657


### Apply a Multi-Layer Perceptron Neural Network with Keras

In [17]:
import keras
from keras.models import Sequential
from keras.layers import Activation, Dense

In [20]:
#define a model
def neural_network(X_train, y_train, X_test, y_test, n):
 #define a model
  model = Sequential()
  model.add(Dense(25, input_dim=n, activation='relu', kernel_initializer='he_uniform'))
  model.add(Dense(10, activation='relu'))
  model.add(Dense(1, activation='linear'))
  # compile a model
  model.compile(loss='mean_squared_logarithmic_error', optimizer=keras.optimizers.Adam(lr=0.01), metrics=['mse'])
  display(model.summary())
  # fit model
  history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=15)
  return (model,history)

def evaluate_model(model, history, X_train, y_train, X_test, y_test):
  _, train_mse = model.evaluate(X_train, y_train, verbose=0)
  _, test_mse = model.evaluate(X_test, y_test, verbose=0)
  print('MSE Train: %.3f, Test: %.3f' % (train_mse, test_mse))

  y_train_predict = model.predict(X_train)
  y_test_predict = model.predict(X_test)
  y_train_r2 = r2_score(y_train, y_train_predict)
  y_test_r2 = r2_score(y_test, y_test_predict)
  print('R2 score: Train: %.3f, Test: %.3f' % (y_train_r2, y_test_r2))

In [21]:
# Defining, compiling and fitting the NN.
model, summary = neural_network(X_train,y_train,X_test,y_test,13)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 25)                350       
_________________________________________________________________
dense_4 (Dense)              (None, 10)                260       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 11        
Total params: 621
Trainable params: 621
Non-trainable params: 0
_________________________________________________________________


None

Epoch 1/15
2830/2830 [==============================] - 5s 2ms/step - loss: 0.0192 - mse: 0.1543 - val_loss: 0.0132 - val_mse: 0.1123
Epoch 2/15
2830/2830 [==============================] - 5s 2ms/step - loss: 0.0124 - mse: 0.1045 - val_loss: 0.0130 - val_mse: 0.1070
Epoch 3/15
2830/2830 [==============================] - 5s 2ms/step - loss: 0.0123 - mse: 0.1047 - val_loss: 0.0129 - val_mse: 0.1077
Epoch 4/15
2830/2830 [==============================] - 5s 2ms/step - loss: 0.0121 - mse: 0.1038 - val_loss: 0.0125 - val_mse: 0.1096
Epoch 5/15
2830/2830 [==============================] - 4s 2ms/step - loss: 0.0123 - mse: 0.1092 - val_loss: 0.0121 - val_mse: 0.1033
Epoch 6/15
2830/2830 [==============================] - 4s 2ms/step - loss: 0.0119 - mse: 0.1010 - val_loss: 0.0141 - val_mse: 0.1177
Epoch 7/15
2830/2830 [==============================] - 5s 2ms/step - loss: 0.0119 - mse: 0.1025 - val_loss: 0.0127 - val_mse: 0.1127
Epoch 8/15
2830/2830 [==============================] - 4s 1ms

In [22]:
evaluate_model(model, summary, X_train, y_train, X_test, y_test)

MSE Train: 0.094, Test: 0.099
R2 score: Train: 0.798, Test: 0.791


### Conclusion 
<br />
1. The models perform better using latent variable weather comparing to results not using latent variables: <br />
model	r2_score	mean_squared_error	mean_absolute_error	time<br />
0	Linear Regression	76.107805	0.108219	0.150529	0.045837 <br />
1	Random Forest	62.857130	0.168238	0.240153	1.976813<br />
2	Decision Tree	61.509206	0.174343	0.245260	0.307280<br />
3	MLP Regressor	77.786256	0.100617	0.144268	9.806735<br />
4	XBoost Regressor	37.678032	0.282286	0.335495	0.707462<br />
5	KNN Regressor	82.910443	0.077407	0.102803	23.215633<br />
On models KNN, it increased 5%. <br />
<br />
2. The Keras MLP didn't perform better than KNN, but still achieved the second best performanance in all the models. Expect to see the different results after optimizing the hyper-variables.